In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statistics as stat
import scipy.stats as scip
from numpy import random as rand
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures as polyfeat
from statsmodels.graphics.gofplots import qqplot
import matplotlib as mpl
from decimal import Decimal

#makes the plot come out in sns format
sns.set()
#read table into python and duration coloumn
table= pd.read_csv('/Users/Windows/Documents/GitHub/Moisture-Sensors/test/centered trimmean.csv')
print(table)
Sensor_val=np.array([table.loc[:,'M0'],table.loc[:,'M1'],table.loc[:,'M2']])
n=len(Sensor_val[0])
#Moisture array based on increments of moisture added
Moisture= np.array([table.loc[:,'Ml']])
Moisture=Moisture[0]
print(Moisture)

In [ ]:
# %%  Polynomial Linear Regression Model function
def PolyRegress(x,y,n,plot,Sensor):
    ##set bias to false so it is not automatically 0
    Poly = polyfeat(degree=n,include_bias= False)
    #using Polynomial Features as required
    PolyFeatures = Poly.fit_transform(x.reshape(-1,1))
    #fit polynomial regression model
    polymodel = LinearRegression()
    polymodel.fit(PolyFeatures, y)
    Pred=polymodel.predict(PolyFeatures)
    ##display model coefficients
    ##print(polymodel.intercept_, polymodel.coef_)
    #Number of parameters being estimated
    q=n+1
    StdDev= stat.stdev(y)
    #Residual sum of squares
    RSS= (np.sum((y-Pred)**2))
    #Log likelihood function with Eqn from Stats book pg 150
    LL=-(len(y)*0.5*np.log(2*np.pi*(StdDev**2)))-(1/(2*StdDev**2))*RSS
    #AIC information
    AIC= 2*q-2*LL
    #Only plot if required
    if plot==True:
        plt.figure()
        plt.scatter(x,y)
        plt.plot(x,Pred,c='r',label='AIC='+str(np.round(AIC,decimals=2))+' RSS='+str(np.round(RSS,decimals=4)))
        plt.xlabel("Menstrual Fluid added (Ml)")
        plt.ylabel("Magnitude of change in Sensor ouput Arb.")
        plt.ticklabel_format(useOffset=False)
        plt.title(Sensor +  " output change against Moisture with a Polynomial regression of Order " + str(n))
        plt.legend()
        plt.show()
        print(n, AIC)
    #Return prediction array
    return Pred


In [ ]:
# %%  Run Polynomial linear regress for different orders to determine best model by AIC
#change order to see the best model
MoisturePred=PolyRegress(Moisture,Sensor_val[0],2,True,"Front Sensor")
#for i in MoisturePred:
    #print(i)
MoisturePred=PolyRegress(Moisture,Sensor_val[1],1,True,"Middle sensor")
#for i in MoisturePred:
    #print(i)
MoisturePred=PolyRegress(Moisture,Sensor_val[2],2,True,"Back Sensor")
#for i in MoisturePred:
    #print(i)

In [ ]:
##checking validity of linear regression assumptions
#chosen model  based on AIC
StdMoisture= (Moisture-np.mean(Moisture))/stat.stdev(Moisture)
MoisturePred= PolyRegress(Moisture,Sensor_val[1],2,True,"Back sensor")
Residuals=Sensor_val[1]-MoisturePred
print(Residuals)
plt.scatter(StdMoisture,Residuals)
plt.xlabel("Standardised Fluid Volume Index")
plt.ylabel("Residual (nm)")
plt.ticklabel_format(useOffset=False)
plt.title("Residuals against Fluid added for a Polynomial regression of order 2 for Middle sensor")
plt.show()
#do a histogram of residuals to see whether it is normally distributed
sns.histplot(Residuals,bins=20,kde=True, stat="probability")
plt.ylabel("Relative Frequency")
plt.xlabel("Residual (nm)")
plt.title('Histogram plot of Residuals')
plt.show()

#Standardise the residuals
Residuals= (Residuals-np.mean(Residuals))/stat.stdev(Residuals)
#qqPLot of the distribution of the residuals versus the Standard normal distribution- more definitive way
#to check whether the residuals are normally distributed than histogram
qqplot(Residuals,line='45')
plt.xlabel('Theoretical')
plt.ylabel('Sample')
plt.title('Normal QQ plot of Residuals')
plt.show()